In [ ]:
from tensorflow.keras import models, Sequential
from tensorflow.keras.layers import Dropout, Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau,ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.metrics import Balanceacc

from models import efficientNetV2B0_model, efficientNetV2B3_model
from config import efficientNet_config

import cv2 as cv
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

import os
# 使用第一張 GPU 卡
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
# pip install opencv-python

In [ ]:
height  = efficientNet_config['height_B0']
width   = efficientNet_config['width_B0']
input_shape  = efficientNet_config['input_shape_B0']

# 讀取資料

In [ ]:
train_dir      = '../dataset_22video_20221031/train'
validation_dir = '../dataset_22video_20221031/validation/'

In [ ]:
train_img_arrays = []
train_img_labels = []
validation_img_arrays = []
validation_img_labels = []

## train
img_paths = glob(train_dir + "/0/*.png")
for img_path in img_paths:
    # imege
    img_array = cv.imread(img_path)
    img_array = cv.resize(img_array,(height,width))     # resize to (224,224)
    train_img_arrays.append(img_array)
    #label
    train_img_labels.append([0])

img_paths = glob(train_dir + "/1/*.png")
for img_path in img_paths:
    # imege
    img_array = cv.imread(img_path)
    img_array = cv.resize(img_array,(height,width))      # resize to (224,224)
    train_img_arrays.append(img_array)
    #label
    train_img_labels.append([1])


## validation
img_paths = glob(validation_dir + "/0/*.png")
for img_path in img_paths:
    # imege
    img_array = cv.imread(img_path)
    img_array = cv.resize(img_array,(height,width))      # resize to (224,224)
    validation_img_arrays.append(img_array)
    #label
    validation_img_labels.append([0])

img_paths = glob(validation_dir + "/1/*.png")
for img_path in img_paths:
    # imege
    img_array = cv.imread(img_path)
    img_array = cv.resize(img_array,(height,width))      # resize to (224,224)
    validation_img_arrays.append(img_array)
    #label
    validation_img_labels.append([1])

In [ ]:
print('訓練集數量= ', len(train_img_labels))

In [ ]:
print('驗證集數量= ',len(validation_img_labels))

In [ ]:
train_img_arrays = np.array(train_img_arrays)
train_img_labels = np.array(train_img_labels)
validation_img_arrays = np.array(validation_img_arrays)
validation_img_labels = np.array(validation_img_labels)

In [ ]:
train_img_arrays, train_img_labels = shuffle(train_img_arrays,train_img_labels)
validation_img_arrays, validation_img_labels = shuffle(validation_img_arrays,validation_img_labels)

In [ ]:
print('訓練集維度= ',train_img_arrays.shape)
print('測試集維度= ',validation_img_arrays.shape)

In [ ]:
model = efficientNetV2B0_model()

model.compile(loss='binary_crossentropy', 
              optimizer=Adam(learning_rate=1e-4),
              metrics=['accuracy', 
                        Balanceacc()
                      ])


In [ ]:
day = '20230213'
checkpoint_filepath = '../model/202211/{}.weights'.format(day)
model_checkpoint_callback = ModelCheckpoint(
        filepath=checkpoint_filepath,
        save_weights_only=True,
        monitor='val_balanceacc',
        mode='max',
        save_best_only=True)

# learning rate 降低
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', 
#                               factor=0.1,
#                               patience=5, 
#                               min_lr=1e-6)



callbacks = [model_checkpoint_callback, 
             #reduce_lr
             ]

In [ ]:
# 訓練
epochs = 1000
history = model.fit(
      x = train_img_arrays,
      y = train_img_labels,
      validation_data = (validation_img_arrays,validation_img_labels),      
      epochs          = epochs,
      verbose         = 1,
      callbacks       = callbacks
)

# 畫圖

In [ ]:
loss         = history.history['loss']
val_loss     = history.history['val_loss']
accuracy     = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
BlA          = history.history['balanceacc']
val_BlA      = history.history['val_balanceacc']

In [ ]:
x = [i for i in range(1, epochs+1)]

min_valloss_x = val_loss.index(min(val_loss)) + 1
min_valloss_y = min(val_loss)

plt.figure(figsize=(24,4))
plt.plot(x, loss, 'r')     
plt.plot(x, val_loss, 'b')     # red dotted line (no marker)

plt.plot(min_valloss_x, min_valloss_y, 'd', color='g')
plt.text(min_valloss_x, min_valloss_y, "({},{})".format(min_valloss_x,round(min_valloss_y,2)), ha='left',va='top',fontsize=20)

plt.legend(['loss','val_loss'])
plt.ylim((0, 1.0))
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title('loss')
plt.savefig('../model/202211/{}_loss.png'.format(day),
            bbox_inches='tight',
            pad_inches=1,
            transparent=False)
plt.show()

In [ ]:
x = [i for i in range(1, epochs+1)]

max_valacc_x = val_accuracy.index(max(val_accuracy)) + 1
max_valacc_y = max(val_accuracy)

plt.figure(figsize=(24,4))
plt.plot(x, accuracy, 'r')     
plt.plot(x, val_accuracy, 'b')     # red dotted line (no marker)

plt.plot(max_valacc_x, max_valacc_y, 'd', color='g')
plt.text(max_valacc_x, max_valacc_y, "({},{})".format(max_valacc_x, round(max_valacc_y,2)), ha='left',va='top',fontsize=20)

plt.legend(['accuracy','val_accuracy'])
plt.ylim((0, 1.1))
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.title('accuracy')
plt.savefig('../model/202211/{}_acc.png'.format(day),
            bbox_inches='tight',
            pad_inches=1,
            transparent=False)
plt.show()

In [ ]:
x = [i for i in range(1, epochs+1)]

max_valBlA_x = val_BlA.index(max(val_BlA)) + 1
max_valBlA_y = max(val_BlA)

plt.figure(figsize=(24,4))
plt.plot(x, BlA, 'r')     
plt.plot(x, val_BlA, 'b')     # red dotted line (no marker)

plt.plot(max_valBlA_x, max_valBlA_y, 'd', color='g')
plt.text(max_valBlA_x, max_valBlA_y, "({},{})".format(max_valBlA_x, round(max_valBlA_y,2)), ha='left',va='top',fontsize=20)

plt.legend(['balance accuracy','val_balance accuracy'])
plt.ylim((0, 1.1))
plt.xlabel('epoch')
plt.ylabel('balance accuracy')
plt.title('balance accuracy')
plt.savefig('../model/202211/{}_BlA.png'.format(day),
            bbox_inches='tight',
            pad_inches=1,
            transparent=False)
plt.show()

In [ ]:
import json
# Get the dictionary containing each metric and the loss for each epoch
history_dict = history.history
# Save it under the form of a json file
json.dump(history_dict, open('../model/202211/{}.history'.format(day), 'w'))